<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/get_ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

!pip install pyscf
!git clone https://github.com/jcandane/CI_Theory

from pyscf import fci, ao2mo, scf, gto
from CI_Theory.CI_combos import MR_X, Ext
from CI_Theory.SlaterCondon import SlaterCondon, givenBgetΛ

     |████████████████████████████████| 37.5 MB 12.1 MB/s 
Cloning into 'CI_Theory'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 14), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (58/58), done.


In [4]:
def get_SO_H1(H_ao, uhf_pyscf):
    Ca, Cb = (uhf_pyscf).mo_coeff
    return np.asarray([np.einsum("AB, Ap, Bq -> pq", H_ao, Ca, Ca), np.einsum("AB, Ap, Bq -> pq", H_ao, Cb, Cb)])

def get_SO_H2(uhf_pyscf):
    Ca, Cb = (uhf_pyscf).mo_coeff
    n = Ca.shape[1]
    eri_aa = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Ca, Ca, Ca), compact=False)).reshape((n,n,n,n), order="C")
    eri_aa -= eri_aa.swapaxes(1,3)
    eri_bb = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Cb, Cb, Cb), compact=False)).reshape((n,n,n,n), order="C")
    eri_bb -= eri_bb.swapaxes(1,3)
    eri_ab = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Ca, Cb, Cb), compact=False)).reshape((n,n,n,n), order="C")
    #eri_ba = (1.*eri_ab).swapaxes(0,3).swapaxes(1,2) ## !! caution depends on symmetry
    eri_ba = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Cb, Ca, Ca), compact=False)).reshape((n,n,n,n), order="C")
    return np.stack(( np.stack((eri_aa, eri_ab)), np.stack((eri_ba, eri_bb)) ))

def get_SO_H(H_ao, uhf_pyscf):
    return get_SO_H1(H_ao, uhf_pyscf), get_SO_H2(uhf_pyscf)

def get_CI_H1(H1_SO, SC1, Binary):
    I_A, J_A, a, a_t, I_B, J_B, b, b_t = SC1

    H1_CI   = np.diag( np.einsum("spp, sIp -> I", H1_SO, Binary) )
    H1_CI[I_A, J_A] += np.einsum("pq, Kp, Kq -> K", H1_SO[0], a_t, a)
    H1_CI[I_B, J_B] += np.einsum("pq, Kp, Kq -> K", H1_SO[1], b_t, b)

    return H1_CI

def get_CI_H2(H2_SO, SC1, SC2, Binary):
    I_A, J_A, a, a_t, I_B, J_B, b, b_t = SC1
    I_AA, J_AA, aa_T, I_AB, J_AB, ab_T, I_BB, J_BB, bb_T, ca, cb = SC2

    H2_CI    = np.diag( np.einsum("stppqq, sIp, tIq -> I", H2_SO, Binary, Binary, optimize=True)/2 )
    H2_CI[I_A, J_A]   = np.einsum("sprqq, Kp, Kr, sKq -> K", H2_SO[0], a_t, a, ca, optimize=True)
    H2_CI[I_B, J_B]   = np.einsum("sprqq, Kp, Kr, sKq -> K", H2_SO[1], b_t, b, cb, optimize=True)
    H2_CI[I_AA, J_AA] = np.einsum("pqrs, Kp, Kq, Kr, Ks -> K", H2_SO[0,0], aa_T[0], aa_T[1], aa_T[2], aa_T[3], optimize=True)
    H2_CI[I_BB, J_BB] = np.einsum("pqrs, Kp, Kq, Kr, Ks -> K", H2_SO[1,1], bb_T[0], bb_T[1], bb_T[2], bb_T[3], optimize=True)
    H2_CI[I_AB, J_AB] = np.einsum("pqrs, Kp, Kq, Kr, Ks -> K", H2_SO[0,1], ab_T[0], ab_T[1], ab_T[2], ab_T[3], optimize=True)

    return H2_CI

def get_CI_H(H1_SO, H2_SO, SC1, SC2, Binary, sign):
    return get_CI_H1(H1_SO, SC1, Binary, sign), get_CI_H2(H2_SO, SC2, Binary, sign)

In [ ]:
    I_A, J_A, A, A_t, I_B, J_B, B, B_t, CA_i, CB_i = SC1

    ll      = np.arange(0, len(A), 1, dtype=int)
    a   = np.zeros((len(A), Binary.shape[2]), dtype=int)
    a_t = np.zeros((len(A), Binary.shape[2]), dtype=int)
    a[ll, A]     = sign[J_A, 0, A]
    a_t[ll, A_t] = sign[I_A, 0, A_t]

    ll      = np.arange(0, len(B), 1, dtype=int)
    b   = np.zeros((len(B), Binary.shape[2]), dtype=int)
    b_t = np.zeros((len(B), Binary.shape[2]), dtype=int)
    b[ll, B]     = sign[J_B, 1, B]
    b_t[ll, B_t] = sign[I_B, 1, B_t] 